# 🏛️ Indian Legal AI Advisor
## Basic Legal Query System

**Author**: DLA Project  
**Date**: February 2026  
**Version**: 1.0 - Basic Query System (Phase 1)

---

## 📋 Overview

This notebook implements a **basic AI-powered legal assistant** trained on Indian laws, case precedents, and legal documents. The system provides legal information, case law search, and preliminary legal advice for the Indian legal system.

**Scope**: This is Phase 1 focusing on basic query answering. Document analysis features are excluded from this version.

### What This System Does

✅ Answer legal queries in natural language (English)  
✅ Search and retrieve relevant case law and statutes  
✅ Provide preliminary legal information with citations  
✅ Explain legal concepts in simple language  
✅ Interactive query interface with ipywidgets  

### Table of Contents

1. [Background & Problem Statement](#section1)
2. [Setup & Dependencies](#section2)
3. [Synthetic Legal Dataset Creation](#section3)
4. [Basic Legal Query System](#section4)
5. [Interactive Query Interface](#section5)
6. [Sample Use Cases](#section6)
7. [Ethical Safeguards & Disclaimers](#section7)


---
<a id="section1"></a>
## 1.0 📚 Background & Problem Statement

### The Access to Justice Problem in India

India faces a severe access to justice crisis:

| Challenge | Statistics |
|-----------|-----------|
| **Population vs Lawyers** | 1.3 billion people with only ~1.7 million lawyers |
| **Case Pendency** | Average 5-15 years in lower courts |
| **Pending Cases** | 3.5 crore (35 million) cases across Indian courts |
| **Cost** | Legal consultation prohibitive for rural/low-income citizens |
| **Language Barriers** | 22+ official languages create communication gaps |

### Why This Matters

- **Limited Legal Awareness**: Most Indians don't know their basic legal rights
- **Cost Barriers**: Initial legal consultation costs ₹2,000-10,000+
- **Time**: Weeks to get appointment with lawyers in major cities
- **Rural Access**: Lawyer availability very limited outside urban areas

### What This System Provides

This AI system aims to:

1. **Democratize Legal Knowledge**: Make legal information accessible to all
2. **Reduce Cost Barriers**: Free preliminary legal information
3. **Save Time**: Instant responses to common legal queries
4. **Educate Citizens**: Explain laws and rights in simple language
5. **Guide Next Steps**: Inform users about legal procedures and options

### System Capabilities (Phase 1)

✅ **Implemented**:
- Legal query answering in natural language
- Case law search using semantic similarity
- Statute lookup and information retrieval
- Interactive query interface
- Sample use cases and demonstrations
- Ethical disclaimers and safeguards

❌ **Not Included** (Future Phases):
- Document analysis (contracts, notices)
- Document generation (legal notices, petitions)
- Multi-lingual support (Hindi, regional languages)
- Advanced RAG with large language models
- Voice interface
- Integration with live legal databases

### Expected Impact

🎯 **Access to Justice**: Enable millions to understand their legal rights  
⚖️ **Cost Savings**: Save ₹2,000-10,000 per preliminary consultation  
⏱️ **Time Savings**: Get answers in seconds vs days/weeks  
📚 **Legal Literacy**: Educate citizens about Indian laws  
🌍 **Rural Reach**: Extend legal awareness beyond urban centers


---
<a id="section2"></a>
## 2.0 🔧 Setup & Dependencies

### Required Libraries

This notebook requires the following Python libraries:
- **transformers**: For NLP models
- **sentence-transformers**: For semantic search
- **faiss-cpu**: For efficient vector similarity search
- **ipywidgets**: For interactive UI
- **numpy, pandas**: For data processing
- **scikit-learn**: For ML utilities


In [ ]:
# Install required packages (run this cell first)
!pip install -q transformers sentence-transformers
!pip install -q faiss-cpu numpy pandas
!pip install -q ipywidgets scikit-learn

print("✅ All packages installed successfully!")


### Import Libraries


In [ ]:
# Standard libraries
import json
import random
import warnings
warnings.filterwarnings('ignore')

# Data processing
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
from datetime import datetime

# ML libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Transformers and semantic search
from sentence_transformers import SentenceTransformer
import faiss

# UI libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

print("✅ All libraries imported successfully!")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


---
<a id="section3"></a>
## 3.0 📊 Synthetic Legal Dataset Creation

Since we cannot scrape live legal databases in this environment, we'll create a comprehensive synthetic dataset that represents real Indian legal content.

### 3.1 Indian Legal Statutes Database


In [ ]:
# Create comprehensive database of Indian legal statutes
INDIAN_STATUTES = {
    "IPC_302": {
        "act": "Indian Penal Code, 1860",
        "section": "302",
        "title": "Punishment for murder",
        "description": "Whoever commits murder shall be punished with death or imprisonment for life, and shall also be liable to fine.",
        "keywords": "murder, homicide, killing, punishment, death penalty, life imprisonment"
    },
    "IPC_420": {
        "act": "Indian Penal Code, 1860",
        "section": "420",
        "title": "Cheating and dishonestly inducing delivery of property",
        "description": "Whoever cheats and thereby dishonestly induces the person deceived to deliver any property to any person, or to make, alter or destroy the whole or any part of a valuable security, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine.",
        "keywords": "cheating, fraud, dishonesty, property, deception, ponzi scheme"
    },
    "IPC_498A": {
        "act": "Indian Penal Code, 1860",
        "section": "498A",
        "title": "Cruelty by husband or relatives of husband",
        "description": "Whoever, being the husband or the relative of the husband of a woman, subjects such woman to cruelty shall be punished with imprisonment up to three years and fine.",
        "keywords": "domestic violence, cruelty, dowry, harassment, matrimonial"
    },
    "CPA_2": {
        "act": "Consumer Protection Act, 2019",
        "section": "2(7)",
        "title": "Definition of Consumer",
        "description": "Consumer means any person who buys any goods for a consideration which has been paid or promised or partly paid and partly promised, or under any system of deferred payment and includes any user of such goods other than the person who buys such goods for consideration paid or promised or partly paid or partly promised, or under any system of deferred payment when such use is made with the approval of such person, but does not include a person who obtains such goods for resale or for any commercial purpose.",
        "keywords": "consumer, buyer, goods, services, commercial, resale"
    },
    "CPA_35": {
        "act": "Consumer Protection Act, 2019",
        "section": "35",
        "title": "Jurisdiction of District Commission",
        "description": "Subject to the other provisions of this Act, the District Commission shall have jurisdiction to entertain complaints where the value of the goods or services and the compensation, if any, claimed does not exceed rupees one crore.",
        "keywords": "jurisdiction, district commission, compensation, complaint, consumer forum"
    },
    "CPA_2_10": {
        "act": "Consumer Protection Act, 2019",
        "section": "2(10)",
        "title": "Defective goods",
        "description": "Goods shall be deemed to be defective when they suffer from any imperfection or shortcoming in quality, quantity, potency, purity or standard which is required to be maintained by law or contract.",
        "keywords": "defective, imperfection, quality, standard, warranty"
    },
    "EASEMENT_7": {
        "act": "Indian Easements Act, 1882",
        "section": "7",
        "title": "Easement of light and air",
        "description": "Where the access and use of light or air to and for any building have been peaceably and openly enjoyed therewith as an easement, and as of right, without interruption, and for twenty years, the easement becomes absolute and indefeasible.",
        "keywords": "easement, light, air, ancient lights, property rights, twenty years"
    },
    "EASEMENT_15": {
        "act": "Indian Easements Act, 1882",
        "section": "15",
        "title": "Acquisition of easement by prescription",
        "description": "An easement may be acquired by virtue of a local custom or by long-continued and uninterrupted user, which is known as a prescriptive easement.",
        "keywords": "prescription, custom, continuous use, property"
    },
    "IDA_25F": {
        "act": "Industrial Disputes Act, 1947",
        "section": "25F",
        "title": "Conditions precedent to retrenchment",
        "description": "No workman employed in any industry who has been in continuous service for not less than one year shall be retrenched by an employer unless: (a) the workman has been given one month's notice in writing indicating the reasons for retrenchment or wages in lieu of such notice; (b) the workman has been paid compensation equivalent to fifteen days average pay for every completed year of continuous service; (c) notice has been served on the appropriate Government or authority.",
        "keywords": "retrenchment, termination, notice, compensation, employment, labor law"
    },
    "IDA_2S": {
        "act": "Industrial Disputes Act, 1947",
        "section": "2(s)",
        "title": "Definition of Workman",
        "description": "Workman means any person employed in any industry to do any manual, unskilled, skilled, technical, operational, clerical or supervisory work for hire or reward.",
        "keywords": "workman, employee, industry, employment"
    },
    "TOPA_55": {
        "act": "Transfer of Property Act, 1882",
        "section": "55",
        "title": "Rights and liabilities of seller",
        "description": "The seller is bound to disclose to the buyer any material defect in the property or in the seller's title thereto of which the seller is, and the buyer is not, aware, and which the buyer could not with ordinary care discover.",
        "keywords": "seller, disclosure, material defect, property sale, caveat emptor, fraud"
    },
    "TOPA_54": {
        "act": "Transfer of Property Act, 1882",
        "section": "54",
        "title": "Sale of immovable property",
        "description": "Sale is a transfer of ownership in exchange for a price paid or promised or part-paid and part-promised.",
        "keywords": "sale, transfer, ownership, price, immovable property"
    },
    "CrPC_154": {
        "act": "Code of Criminal Procedure, 1973",
        "section": "154",
        "title": "Information in cognizable cases",
        "description": "Every information relating to the commission of a cognizable offence, if given orally to an officer in charge of a police station, shall be reduced to writing by him or under his direction, and shall be read over to the informant; and every such information, whether given in writing or reduced to writing, shall be signed by the person giving it.",
        "keywords": "FIR, police, cognizable, information, complaint"
    },
    "CrPC_156_3": {
        "act": "Code of Criminal Procedure, 1973",
        "section": "156(3)",
        "title": "Power of Magistrate to order investigation",
        "description": "A Magistrate empowered may order such an investigation to be made by a police officer as he thinks necessary.",
        "keywords": "magistrate, investigation, police, FIR refusal"
    },
    "CrPC_41": {
        "act": "Code of Criminal Procedure, 1973",
        "section": "41",
        "title": "When police may arrest without warrant",
        "description": "Any police officer may without warrant arrest any person who has been concerned in any cognizable offence or against whom a reasonable complaint has been made.",
        "keywords": "arrest, warrant, cognizable, police power"
    },
    "RTI_6": {
        "act": "Right to Information Act, 2005",
        "section": "6",
        "title": "Request for obtaining information",
        "description": "A person who desires to obtain any information under this Act shall make a request in writing or through electronic means in English or Hindi or in the official language of the area.",
        "keywords": "RTI, information, request, transparency, government"
    }
}

print(f"✅ Created database of {len(INDIAN_STATUTES)} Indian legal statutes")
print(f"\nStatutes covered:")
print(f"  • Indian Penal Code (IPC)")
print(f"  • Consumer Protection Act (CPA)")
print(f"  • Industrial Disputes Act (IDA)")
print(f"  • Transfer of Property Act (TOPA)")
print(f"  • Indian Easements Act")
print(f"  • Code of Criminal Procedure (CrPC)")
print(f"  • Right to Information Act (RTI)")
print(f"\nSample Statutes:")
for i, (key, statute) in enumerate(list(INDIAN_STATUTES.items())[:3]):
    print(f"  {i+1}. {statute['act']}, Section {statute['section']}: {statute['title']}")


### 3.2 Sample Indian Case Law Database

Creating a synthetic database of landmark and representative Indian legal cases across multiple domains.


In [ ]:
# Create comprehensive case law database
INDIAN_CASES = [
    {
        "case_id": "SC_2021_001",
        "case_name": "Raj Traders v. State Consumer Forum",
        "citation": "(2021) 5 SCC 234",
        "court": "Supreme Court of India",
        "year": 2021,
        "judges": ["Justice A. Kumar", "Justice B. Sharma"],
        "acts": ["Consumer Protection Act 2019 - Section 2(7)", "Section 35"],
        "domain": "consumer_law",
        "facts": "The appellant purchased an electronic device from an e-commerce platform which developed defects within the warranty period of 6 months. The seller refused refund or replacement claiming misuse by the customer without providing any evidence. The device had manufacturing defects as per independent technical evaluation.",
        "legal_issues": ["Definition of defective goods", "Consumer rights under warranty", "Burden of proof in defect cases", "E-commerce platform liability"],
        "judgment": "The Court held that once a defect appears within warranty period, the burden of proving misuse lies on the seller, not the consumer. Consumer entitled to refund or replacement as per Consumer Protection Act. E-commerce platforms also held jointly and severally liable for defective products sold through their platform as they facilitate the transaction and earn commission.",
        "precedent": "Established that online marketplaces share liability with sellers for defective products. Shifted burden of proof to seller in warranty period defect cases.",
        "keywords": "consumer protection, defective goods, warranty, e-commerce liability, burden of proof"
    },
    {
        "case_id": "SC_2020_045",
        "case_name": "Sunita Devi v. Rajeev Properties Ltd",
        "citation": "(2020) 3 SCC 456",
        "court": "Supreme Court of India",
        "year": 2020,
        "judges": ["Justice C. Reddy", "Justice D. Patel"],
        "acts": ["Transfer of Property Act 1882 - Section 55"],
        "domain": "property_law",
        "facts": "Seller sold residential property to buyer without disclosing that: (a) the property had significant structural defects in foundation, (b) there was pending litigation regarding property title from previous owner, (c) the property was subject to municipal demolition notice for unauthorized construction. Buyer discovered these facts 6 months after purchase and sought rescission of sale deed.",
        "legal_issues": ["Seller's duty of disclosure under TOPA Section 55", "Material defects - what constitutes", "Right to rescind contract for non-disclosure", "Fraudulent misrepresentation"],
        "judgment": "Court ruled in favor of buyer. Seller has statutory duty under Section 55 TOPA to disclose all material defects. Non-disclosure of structural defects and pending litigation constitutes fraudulent misrepresentation. Principle of caveat emptor does not apply when seller actively conceals material facts. Sale contract rescinded, buyer entitled to full refund of purchase price with interest at 9% per annum from date of sale.",
        "precedent": "Strengthened seller's disclosure obligations; defined material defects broadly; confirmed pending litigation is material fact requiring disclosure.",
        "keywords": "property sale, disclosure duty, material defect, rescission, fraud, caveat emptor"
    },
    {
        "case_id": "SC_2019_089",
        "case_name": "Municipal Corporation of Delhi v. Ram Singh",
        "citation": "(2019) 2 SCC 789",
        "court": "Supreme Court of India",
        "year": 2019,
        "judges": ["Justice E. Gupta"],
        "acts": ["Indian Easements Act 1882 - Section 7"],
        "domain": "property_law",
        "facts": "Respondent Ram Singh owned a house that had enjoyed unobstructed sunlight from the southern side for 28 years. His neighbor constructed a 3-story building with a high boundary wall that completely blocked sunlight to respondent's property. Respondent filed suit seeking mandatory injunction for partial demolition to restore light access.",
        "legal_issues": ["Easement of light and air - Section 7", "Ancient lights doctrine in India", "Twenty year prescription period", "Actionable obstruction", "Balance of convenience"],
        "judgment": "Court held that right to light is acquired after 20 years of uninterrupted enjoyment as per Section 7 of Indian Easements Act. Having enjoyed unobstructed light for 28 years, respondent acquired indefeasible easement right. Neighbor's construction violates this easement. However, complete demolition would be disproportionate. Court ordered partial demolition of upper floor and setback of boundary wall to restore reasonable quantum of light, while allowing neighbor to retain ground floor structure.",
        "precedent": "Confirmed that easement of light is acquired after 20 years under Section 7. Established principle of reasonable light access (not absolute).",
        "keywords": "easement, light and air, ancient lights, property rights, prescription, twenty years"
    },
    {
        "case_id": "SC_2018_112",
        "case_name": "Workmen Union v. ABC Manufacturing Ltd",
        "citation": "(2018) 4 SCC 123",
        "court": "Supreme Court of India",
        "year": 2018,
        "judges": ["Justice F. Iyer", "Justice G. Nair"],
        "acts": ["Industrial Disputes Act 1947 - Section 25F"],
        "domain": "labor_law",
        "facts": "Company terminated 50 workmen who had 2-5 years of service, citing financial losses and need for restructuring. Termination was done immediately without any notice, compensation, or notice to government. Company claimed exemption citing financial emergency. Workers challenged termination before Labor Court.",
        "legal_issues": ["Compliance with Section 25F IDA", "Notice requirement in retrenchment", "Retrenchment compensation calculation", "Financial distress as defense", "Back wages"],
        "judgment": "Court ruled termination completely void as it violated all three requirements of Section 25F: (a) no notice or pay in lieu given, (b) no retrenchment compensation paid, (c) no notice to appropriate government. Financial distress does not waive mandatory statutory requirements. Workers entitled to: (1) reinstatement with continuity of service, (2) back wages for entire period of unemployment, (3) retrenchment compensation of 15 days wages × years of service if they opt for separation instead of reinstatement.",
        "precedent": "Established strict compliance requirement for Section 25F; financial difficulties don't exempt employer from statutory obligations.",
        "keywords": "retrenchment, termination, labor rights, compensation, notice, back wages, Section 25F"
    },
    {
        "case_id": "HC_DEL_2021_234",
        "case_name": "Sharma Enterprises v. Verma Builders",
        "citation": "2021 DHC 567",
        "court": "Delhi High Court",
        "year": 2021,
        "judges": ["Justice H. Mehta"],
        "acts": ["Transfer of Property Act 1882", "Indian Contract Act 1872"],
        "domain": "property_law",
        "facts": "Builder agreed to deliver possession of apartment within 36 months from booking date. Actual delivery happened after 62 months - a delay of 26 months. Buyer paid full consideration as per payment plan. Agreement had no explicit penalty clause for delayed possession. Buyer sought compensation for delayed possession and rental loss.",
        "legal_issues": ["Delay in possession", "Liquidated damages vs penalty", "Implied terms", "Consequential damages", "Buyer's remedies"],
        "judgment": "Court held builder liable for inordinate delay. Even without explicit penalty clause, builder must compensate for delay as it amounts to breach of contract. Compensation awarded at rate of ₹7.50 per sq ft per month (prevailing rental rate in area) for entire delay period. Additionally awarded litigation costs of ₹50,000. Specific performance ordered directing immediate possession with compensation for delay.",
        "precedent": "Established that builders must compensate for possession delays even without explicit penalty clause; compensation linked to prevailing rental rates.",
        "keywords": "property delay, builder liability, possession, compensation, breach of contract"
    },
    {
        "case_id": "SC_2022_078",
        "case_name": "State of Maharashtra v. Kumar Traders",
        "citation": "(2022) 1 SCC 890",
        "court": "Supreme Court of India",
        "year": 2022,
        "judges": ["Justice I. Khanna", "Justice J. Bose"],
        "acts": ["Indian Penal Code 1860 - Section 420", "Section 34"],
        "domain": "criminal_law",
        "facts": "Accused operated a Ponzi scheme promising 30% monthly returns on investments in agricultural commodities trading. Attracted 5,000 investors who collectively invested ₹200 crores. Scheme collapsed after 14 months, causing massive losses. Accused used new investor money to pay earlier investors (classic Ponzi structure) and diverted significant funds for personal use.",
        "legal_issues": ["Cheating under IPC 420", "Dishonest inducement", "Ponzi scheme legality", "Intention to defraud", "Restitution to victims"],
        "judgment": "Court convicted all accused under Section 420 IPC for cheating and criminal conspiracy. Held that promise of unrealistic returns (30% monthly = 360% annually) with no genuine business model constitutes dishonest inducement from inception. Sentence: 7 years rigorous imprisonment and fine of ₹10 crores. Ordered attachment of all properties of accused and restitution to defrauded investors on pro-rata basis from realized assets.",
        "precedent": "Established that Ponzi schemes constitute cheating under IPC 420 from inception; restitution is integral part of remedy for fraud victims.",
        "keywords": "cheating, fraud, ponzi scheme, IPC 420, dishonest inducement, restitution"
    },
    {
        "case_id": "SC_2019_156",
        "case_name": "Consumer Forum v. National Telecom Service Provider",
        "citation": "(2019) 6 SCC 345",
        "court": "Supreme Court of India",
        "year": 2019,
        "judges": ["Justice K. Singh"],
        "acts": ["Consumer Protection Act 2019 - Section 2(7)", "TRAI Regulations"],
        "domain": "consumer_law",
        "facts": "Customer requested disconnection of mobile service in January through proper channel. Company acknowledged request. However, company continued billing for 6 months after disconnection date, generating bills totaling ₹15,800 for a service customer was not using. Company also reported customer as defaulter to credit bureau, affecting credit score.",
        "legal_issues": ["Service deficiency", "Unfair trade practice", "Post-cancellation billing", "Defamation through credit bureau reporting", "Compensation for harassment"],
        "judgment": "Court held continuing to bill after valid disconnection request is clear deficiency in service and unfair trade practice. Company's conduct caused financial loss, mental harassment, and credit score damage. Ordered: (1) refund of all wrongly charged amounts (₹15,800), (2) compensation for harassment (₹50,000), (3) correction of credit bureau records, (4) litigation costs (₹25,000). Warned service providers must honor disconnection requests immediately.",
        "precedent": "Service providers liable for post-cancellation billing; established compensation for credit score damage due to wrongful reporting.",
        "keywords": "service deficiency, unfair practice, telecom, billing, harassment, credit bureau"
    },
    {
        "case_id": "HC_BOM_2020_445",
        "case_name": "Patel v. ABC Housing Society",
        "citation": "2020 BHC 789",
        "court": "Bombay High Court",
        "year": 2020,
        "judges": ["Justice L. Desai"],
        "acts": ["Maharashtra Ownership Flats Act 1963", "Maharashtra Cooperative Societies Act"],
        "domain": "property_law",
        "facts": "Flat owner wanted to do internal renovation including: (a) changing flooring, (b) modifying bathroom layout, (c) adding built-in wardrobes. No structural changes proposed. Society Managing Committee denied permission citing 'internal guidelines' without specific reasons. Owner challenged arbitrary denial.",
        "legal_issues": ["Housing society's regulatory powers", "Flat owner's rights", "Reasonableness of restrictions", "Internal vs structural alterations"],
        "judgment": "Court held society can regulate alterations to maintain uniformity and structural safety, but cannot arbitrarily deny permission for internal non-structural changes. Proposed renovations were purely internal and aesthetic, not affecting structure or other members. Outright denial without specific reasons is unreasonable exercise of power. Permission granted with conditions: (1) no structural changes, (2) work during permitted hours, (3) proper debris disposal, (4) restoration of common areas if damaged.",
        "precedent": "Housing societies cannot arbitrarily deny permission for internal non-structural renovations; restrictions must be reasonable and specific.",
        "keywords": "housing society, renovation, flat owner rights, internal alterations, reasonableness"
    },
    {
        "case_id": "SC_2020_199",
        "case_name": "Domestic Violence Victim v. State",
        "citation": "(2020) 7 SCC 123",
        "court": "Supreme Court of India",
        "year": 2020,
        "judges": ["Justice M. Chandrachud"],
        "acts": ["Protection of Women from Domestic Violence Act 2005", "IPC Section 498A"],
        "domain": "family_law",
        "facts": "Woman subjected to continuous physical and mental cruelty by husband and in-laws for dowry. Multiple instances of abuse documented through medical records. Woman filed complaint under Protection of Women from Domestic Violence Act.",
        "legal_issues": ["Domestic violence definition", "Protection orders", "Interim relief", "Monetary relief"],
        "judgment": "Court granted comprehensive protection order including: (1) residence order allowing woman to reside in shared household, (2) protection order restraining respondents from committing acts of violence, (3) monetary relief for medical expenses and maintenance, (4) custody of minor children to woman. Criminal proceedings under IPC 498A directed to continue separately.",
        "precedent": "Strengthened interpretation of domestic violence to include economic and emotional abuse; established right to reside in matrimonial home.",
        "keywords": "domestic violence, dowry, cruelty, protection order, women rights, IPC 498A"
    },
    {
        "case_id": "SC_2021_267",
        "case_name": "RTI Applicant v. Public Authority",
        "citation": "(2021) 4 SCC 678",
        "court": "Supreme Court of India",
        "year": 2021,
        "judges": ["Justice N. Ramana"],
        "acts": ["Right to Information Act 2005 - Section 6"],
        "domain": "constitutional_law",
        "facts": "Citizen filed RTI application seeking information about government tender process. Public authority rejected application citing 'third party commercial information' exemption. Information Commissioner upheld rejection.",
        "legal_issues": ["Scope of RTI Act", "Exemptions applicability", "Public interest override", "Commercial confidence vs transparency"],
        "judgment": "Court held that exemptions under RTI Act must be narrowly interpreted. Information about government procurement process is not commercial confidence of third parties but public information. Transparency in public procurement serves larger public interest and prevents corruption. Order to disclose information related to evaluation criteria, bidding process, and award decision, while protecting specific technical/financial data of unsuccessful bidders.",
        "precedent": "Narrow interpretation of RTI exemptions; public procurement information must be disclosed in public interest.",
        "keywords": "RTI, right to information, transparency, public procurement, exemptions"
    }
]

print(f"✅ Created database of {len(INDIAN_CASES)} landmark case laws")
print(f"\nDomains covered:")
domains = set(case['domain'] for case in INDIAN_CASES)
for domain in sorted(domains):
    count = sum(1 for case in INDIAN_CASES if case['domain'] == domain)
    print(f"  • {domain}: {count} cases")

print(f"\nSample Cases:")
for i, case in enumerate(INDIAN_CASES[:3], 1):
    print(f"  {i}. {case['case_name']} {case['citation']}")
    print(f"     {case['court']} ({case['year']}) - {case['domain']}")


### 3.3 Legal Q&A Dataset

Creating Q&A pairs for common legal queries that Indians face.


In [ ]:
# Legal Q&A pairs for common queries
LEGAL_QA_PAIRS = [
    {
        "question": "Can I file a complaint if a product I bought online is defective?",
        "answer": "Yes! Under the Consumer Protection Act 2019, you can file a complaint if goods are defective. You are protected as a 'consumer' under Section 2(7). File with the District Consumer Forum if claim is under ₹1 crore (Section 35). You can claim refund, replacement, or compensation. Both the seller and the e-commerce platform can be held liable.",
        "domain": "consumer_law",
        "related_statutes": ["CPA_2", "CPA_35", "CPA_2_10"],
        "related_cases": ["SC_2021_001"]
    },
    {
        "question": "My landlord is not returning my security deposit. What can I do?",
        "answer": "You have several options: (1) Send a legal notice demanding return with specific timeline (usually 15-30 days), (2) If rental agreement exists, file suit for recovery based on contract terms, (3) File consumer complaint if you can establish landlord-tenant as service relationship, (4) Approach Small Causes Court for amounts up to ₹3 lakhs (varies by state). Keep copies of rent agreement, deposit receipts, and correspondence as evidence.",
        "domain": "civil_law",
        "related_statutes": [],
        "related_cases": []
    },
    {
        "question": "Can my employer terminate me without notice?",
        "answer": "NO, if you've worked for 1+ years in an industrial establishment. Under Section 25F of Industrial Disputes Act 1947, employer MUST: (a) Give one month notice in writing or pay wages in lieu, (b) Pay retrenchment compensation (15 days average pay × completed years of service), (c) Notify appropriate government. Termination without following these is illegal and void. You can approach Labor Court for reinstatement and back wages.",
        "domain": "labor_law",
        "related_statutes": ["IDA_25F", "IDA_2S"],
        "related_cases": ["SC_2018_112"]
    },
    {
        "question": "Seller didn't tell me about property defects. Can I cancel the sale?",
        "answer": "YES! Under Section 55 of Transfer of Property Act 1882, seller has statutory duty to disclose all material defects in property or title that seller knows but buyer doesn't, and buyer couldn't discover with ordinary care. Non-disclosure constitutes fraud. You can file civil suit for: (1) Rescission (cancellation) of sale deed, (2) Refund of purchase price with interest, (3) Damages for loss suffered. Act within limitation period of 3 years from discovery of fraud.",
        "domain": "property_law",
        "related_statutes": ["TOPA_55", "TOPA_54"],
        "related_cases": ["SC_2020_045"]
    },
    {
        "question": "My neighbor built a wall blocking sunlight to my house. Is this legal?",
        "answer": "If you've enjoyed unobstructed sunlight for 20+ years, you have acquired 'easement of light' under Section 7 of Indian Easements Act 1882. This becomes an indefeasible right. Your neighbor's obstruction is actionable. Steps: (1) Send legal notice citing Section 7 and your 20+ years enjoyment, (2) Try mediation/negotiation, (3) If unsuccessful, file civil suit for mandatory injunction for partial removal/modification. Court will balance rights and order reasonable light restoration.",
        "domain": "property_law",
        "related_statutes": ["EASEMENT_7", "EASEMENT_15"],
        "related_cases": ["SC_2019_089"]
    },
    {
        "question": "What is IPC Section 420 and when does it apply?",
        "answer": "IPC Section 420 deals with 'Cheating and dishonestly inducing delivery of property'. Elements: (1) Deception/cheating, (2) Dishonest intention, (3) Inducing victim to deliver property/valuables. Punishment: imprisonment up to 7 years + fine. Common in: fraud, ponzi schemes, fake investment schemes, bounced cheques with dishonest intent, impersonation fraud. To file complaint: Go to police station, file FIR under CrPC Section 154. If police refuse, approach Magistrate under CrPC Section 156(3).",
        "domain": "criminal_law",
        "related_statutes": ["IPC_420", "CrPC_154", "CrPC_156_3"],
        "related_cases": ["SC_2022_078"]
    },
    {
        "question": "How do I file an FIR for a cognizable offence?",
        "answer": "Under Section 154 CrPC: (1) Go to police station in whose jurisdiction crime occurred, (2) Give information orally or in writing to officer-in-charge, (3) Police MUST register FIR for cognizable offences (serious crimes like theft, assault, cheating), (4) FIR will be reduced to writing, read back to you, and you sign it, (5) You get free copy of FIR. If police refuse to register FIR: (1) Note officer's name and badge number, (2) Send written complaint via registered post to Superintendent of Police, (3) Approach Magistrate under Section 156(3) CrPC to direct police investigation.",
        "domain": "criminal_law",
        "related_statutes": ["CrPC_154", "CrPC_156_3", "CrPC_41"],
        "related_cases": []
    },
    {
        "question": "Is online marketplace liable for defective products sold on their platform?",
        "answer": "YES! Recent Supreme Court precedent in Raj Traders v. State Consumer Forum (2021) 5 SCC 234 established that e-commerce platforms share joint and several liability with sellers for defective products. Rationale: They facilitate transaction, earn commission, and hold themselves as providing marketplace service. Consumers can file complaints against BOTH seller and platform under Consumer Protection Act 2019. Platform cannot escape liability by claiming to be mere intermediary.",
        "domain": "consumer_law",
        "related_statutes": ["CPA_2", "CPA_35", "CPA_2_10"],
        "related_cases": ["SC_2021_001"]
    },
    {
        "question": "What compensation can I get if my employer terminates me illegally?",
        "answer": "If termination violates Section 25F of Industrial Disputes Act, you are entitled to: (1) REINSTATEMENT to your job with full back wages for entire period of illegal termination (from termination date to reinstatement), (2) CONTINUITY OF SERVICE - period of unemployment counted as service, (3) If reinstatement not viable (company closed/downsized), lump sum compensation instead of back wages, (4) Retrenchment compensation (15 days wages × years of service) that should have been paid. File complaint with Labor Commissioner (free) or directly to Labor Court within 3 years.",
        "domain": "labor_law",
        "related_statutes": ["IDA_25F"],
        "related_cases": ["SC_2018_112"]
    },
    {
        "question": "Builder delayed giving me possession. Can I claim compensation?",
        "answer": "YES! Even without explicit penalty clause in agreement, you can claim compensation for delayed possession. Courts recognize it as breach of contract. Compensation typically calculated at: (1) Prevailing rental rate in locality (per sq ft per month), or (2) If agreement has clause for buyer's delayed payment, same rate applies to builder's delay. You can also seek: (1) Specific performance (forcing delivery of possession), (2) Interest on amounts paid, (3) Litigation costs. File complaint with Consumer Forum (if residential purpose) or civil court. Keep payment receipts, agreement copy, correspondence as evidence.",
        "domain": "property_law",
        "related_statutes": ["TOPA_55"],
        "related_cases": ["HC_DEL_2021_234"]
    },
    {
        "question": "What information can I get under RTI Act?",
        "answer": "Under Right to Information Act 2005, you can get ANY information held by public authorities (government departments, PSUs, bodies funded by government) EXCEPT: (1) Information affecting sovereignty, security, strategic interests, (2) Cabinet deliberations, (3) Trade secrets, commercial confidence (limited scope), (4) Personal information not serving public interest. Process: File application under Section 6 to Public Information Officer (PIO) of concerned department, pay ₹10 fee, get response within 30 days. If denied or no response, appeal to Appellate Authority, then Information Commission.",
        "domain": "constitutional_law",
        "related_statutes": ["RTI_6"],
        "related_cases": ["SC_2021_267"]
    },
    {
        "question": "Can housing society deny permission for internal renovation?",
        "answer": "NO, not arbitrarily. While society has power to regulate alterations for structural safety and uniformity, they cannot unreasonably deny permission for purely internal, non-structural changes. Your rights: (1) Internal modifications (flooring, painting, fixtures) are your right as owner, (2) Society can impose reasonable conditions (working hours, debris disposal, no structural changes), (3) For structural changes, you need society permission and must comply with building bylaws. If denied: Send written application detailing proposed work, if still denied arbitrarily, approach civil court. Recent precedent: Bombay HC in Patel v. ABC Housing Society (2020) held societies cannot arbitrarily deny internal renovations.",
        "domain": "property_law",
        "related_statutes": [],
        "related_cases": ["HC_BOM_2020_445"]
    },
    {
        "question": "What should I do if facing domestic violence?",
        "answer": "Immediate steps: (1) SAFETY FIRST - Move to safe location if in danger, (2) File complaint under Protection of Women from Domestic Violence Act 2005 at nearest police station or Protection Officer, (3) Simultaneously file under IPC 498A if cruelty for dowry. Reliefs available: (1) Protection order restraining abuser from violence, (2) Residence order - right to live in shared household, (3) Monetary relief for medical expenses, maintenance, (4) Custody of children, (5) Compensation for injuries. For free legal aid: Contact District Legal Services Authority. Emergency: Call Women Helpline 1091 or National Commission for Women helpline. Document evidence: Medical reports, photos of injuries, witnesses.",
        "domain": "family_law",
        "related_statutes": ["IPC_498A"],
        "related_cases": ["SC_2020_199"]
    }
]

print(f"✅ Created {len(LEGAL_QA_PAIRS)} legal Q&A pairs")
print(f"\nDomains covered: {len(set(qa['domain'] for qa in LEGAL_QA_PAIRS))} domains")
print(f"\nSample Questions:")
for i, qa in enumerate(LEGAL_QA_PAIRS[:4], 1):
    print(f"  {i}. {qa['question'][:70]}...")
    print(f"     Domain: {qa['domain']}")


---
<a id="section4"></a>
## 4.0 🤖 Basic Legal Query System

Now we'll build the query system using:
1. **Semantic Search** - Find relevant cases using sentence embeddings
2. **Keyword Matching** - Find applicable statutes
3. **Q&A Matching** - Match direct questions with answers

### 4.1 Initialize Semantic Search Model


In [ ]:
# Load sentence transformer model for semantic search
print("Loading sentence transformer model (this may take a minute)...")
semantic_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("✅ Model loaded successfully!")

# Create embeddings for case law database
print("\nCreating embeddings for case law database...")
case_texts = [
    f"{case['case_name']}. Facts: {case['facts']} Issues: {' '.join(case['legal_issues'])} Judgment: {case['judgment']} Keywords: {case['keywords']}"
    for case in INDIAN_CASES
]
case_embeddings = semantic_model.encode(case_texts, show_progress_bar=True)
print(f"✅ Created embeddings for {len(case_texts)} cases (dimension: {case_embeddings.shape[1]})")

# Create FAISS index for fast similarity search
dimension = case_embeddings.shape[1]
case_index = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity
# Normalize embeddings for cosine similarity
faiss.normalize_L2(case_embeddings)
case_index.add(case_embeddings)
print(f"✅ FAISS index created with {case_index.ntotal} vectors")
print(f"   Index type: Inner Product (cosine similarity)")


### 4.2 Search Functions

Implementing search across all data sources.


In [ ]:
def search_cases(query: str, top_k: int = 3) -> List[Dict]:
    """
    Search for relevant cases using semantic similarity
    
    Args:
        query: User's legal query
        top_k: Number of top results to return
    
    Returns:
        List of relevant cases with relevance scores
    """
    # Encode query
    query_embedding = semantic_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Search in FAISS index
    scores, indices = case_index.search(query_embedding, top_k)
    
    # Build results
    results = []
    for idx, score in zip(indices[0], scores[0]):
        case = INDIAN_CASES[idx].copy()
        case['relevance_score'] = float(score)
        results.append(case)
    
    return results


def search_statutes(query: str) -> List[Dict]:
    """
    Search for relevant statutes using keyword matching
    
    Args:
        query: User's legal query
    
    Returns:
        List of relevant statutes sorted by relevance
    """
    query_lower = query.lower()
    results = []
    
    for key, statute in INDIAN_STATUTES.items():
        score = 0
        
        # Direct section match (high weight)
        if statute['section'].lower() in query_lower:
            score += 5
        
        # Act name match (high weight)
        act_words = statute['act'].lower().split()
        if any(word in query_lower for word in act_words if len(word) > 3):
            score += 3
        
        # Title and description keyword matches
        keywords_text = f"{statute['title']} {statute['description']} {statute['keywords']}".lower()
        query_words = [w for w in query_lower.split() if len(w) > 3]
        for word in query_words:
            if word in keywords_text:
                score += 1
        
        if score > 0:
            statute_copy = statute.copy()
            statute_copy['key'] = key
            statute_copy['relevance_score'] = score
            results.append(statute_copy)
    
    # Sort by relevance
    results.sort(key=lambda x: x['relevance_score'], reverse=True)
    return results[:5]  # Return top 5


def search_qa(query: str, threshold: float = 0.5) -> List[Dict]:
    """
    Search for matching Q&A pairs
    
    Args:
        query: User's legal query
        threshold: Minimum similarity threshold
    
    Returns:
        List of matching Q&A pairs above threshold
    """
    # Create embeddings for Q&A questions
    qa_questions = [qa['question'] for qa in LEGAL_QA_PAIRS]
    qa_embeddings = semantic_model.encode(qa_questions)
    faiss.normalize_L2(qa_embeddings)
    
    # Encode query
    query_embedding = semantic_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Compute similarities
    similarities = np.dot(qa_embeddings, query_embedding.T).flatten()
    
    # Get matches above threshold
    results = []
    for i, sim in enumerate(similarities):
        if sim > threshold:
            qa_copy = LEGAL_QA_PAIRS[i].copy()
            qa_copy['similarity'] = float(sim)
            results.append(qa_copy)
    
    # Sort by similarity
    results.sort(key=lambda x: x['similarity'], reverse=True)
    return results[:3]  # Return top 3


# Test search functions
print("✅ Search functions initialized")
print("\nTesting search functions...")

test_query = "Can I sue if product is defective?"
print(f"\nTest query: '{test_query}'")
print(f"  Found {len(search_cases(test_query, top_k=2))} relevant cases")
print(f"  Found {len(search_statutes(test_query))} relevant statutes")
print(f"  Found {len(search_qa(test_query))} matching Q&A pairs")


### 4.3 Legal Advisor Class

Main class that integrates all components to provide comprehensive legal information.


In [ ]:
class IndianLegalAdvisor:
    """
    Main class for the Indian Legal AI Advisor
    Integrates case search, statute lookup, and Q&A matching
    """
    
    def __init__(self):
        self.cases = INDIAN_CASES
        self.statutes = INDIAN_STATUTES
        self.qa_pairs = LEGAL_QA_PAIRS
        print("✅ Indian Legal Advisor initialized")
        
    def process_query(self, query: str) -> Dict:
        """
        Process a legal query and return comprehensive response
        
        Args:
            query: User's legal question
        
        Returns:
            Dict containing matching Q&A, statutes, cases, and advice
        """
        # Search all sources
        relevant_cases = search_cases(query, top_k=3)
        relevant_statutes = search_statutes(query)
        matching_qa = search_qa(query, threshold=0.5)
        
        # Build response
        response = {
            'query': query,
            'matching_qa': matching_qa,
            'relevant_statutes': relevant_statutes,
            'relevant_cases': relevant_cases,
            'advice': self._generate_advice(query, matching_qa, relevant_statutes, relevant_cases)
        }
        
        return response
    
    def _generate_advice(self, query: str, qa_matches: List, statutes: List, cases: List) -> str:
        """
        Generate legal advice based on retrieved information
        Uses a simple rule-based approach for Phase 1
        """
        advice_parts = []
        
        # If we have a strong Q&A match, use it
        if qa_matches and qa_matches[0]['similarity'] > 0.7:
            best_match = qa_matches[0]
            advice_parts.append(f"**Direct Answer:**\n{best_match['answer']}\n")
        
        # Add statute information
        if statutes:
            advice_parts.append("**📚 Applicable Laws:**")
            for statute in statutes[:3]:  # Top 3 statutes
                advice_parts.append(f"\n• **{statute['act']}, Section {statute['section']}**: {statute['title']}")
                advice_parts.append(f"  _{statute['description']}_")
        
        # Add case law
        if cases and cases[0]['relevance_score'] > 0.3:  # Minimum relevance threshold
            advice_parts.append("\n\n**⚖️ Relevant Case Law:**")
            for case in cases[:2]:  # Top 2 cases
                advice_parts.append(f"\n• **{case['case_name']}** {case['citation']}")
                advice_parts.append(f"  Court: {case['court']} ({case['year']})")
                advice_parts.append(f"  Precedent: _{case['precedent']}_")
        
        # Add next steps if we have good information
        if qa_matches or statutes or (cases and cases[0]['relevance_score'] > 0.3):
            advice_parts.append("\n\n**🎯 Suggested Next Steps:**")
            advice_parts.append("1. Consult with a qualified lawyer for advice specific to your situation")
            advice_parts.append("2. Gather all relevant documents and evidence")
            advice_parts.append("3. Note down timeline of events")
            advice_parts.append("4. Check limitation periods for legal action")
        
        # If nothing found
        if not advice_parts:
            advice_parts.append("I don't have specific information about this query in my database.")
            advice_parts.append("This may be a specialized legal matter requiring consultation with a lawyer.")
            advice_parts.append("\nYou can:")
            advice_parts.append("• Consult a lawyer specializing in this area")
            advice_parts.append("• Contact your District Legal Services Authority for free legal aid")
            advice_parts.append("• Visit your state's law department website for information")
        
        return "\n".join(advice_parts)
    
    def format_response_html(self, response: Dict) -> str:
        """
        Format response as HTML for nice display in Jupyter
        """
        # Convert markdown-style formatting to HTML
        advice_html = response['advice'].replace('**', '<strong>').replace('**', '</strong>')
        advice_html = advice_html.replace('_', '<em>').replace('_', '</em>')
        advice_html = advice_html.replace('\n', '<br>')
        
        html = f"""
        <div style="border: 2px solid #2196F3; padding: 20px; border-radius: 10px; background-color: #f8f9fa; margin: 10px 0;">
            <h3 style="color: #2196F3; margin-top: 0;">�� Legal Information</h3>
            
            <div style="background: white; padding: 15px; border-radius: 5px; margin-bottom: 15px;">
                <strong>Your Query:</strong><br>
                <span style="font-size: 16px;">{response['query']}</span>
            </div>
            
            <div style="background: white; padding: 15px; border-radius: 5px; margin-bottom: 15px;">
                <h4 style="color: #333; margin-top: 0;">Response:</h4>
                <div style="line-height: 1.8;">{advice_html}</div>
            </div>
            
            <div style="background: #fff3cd; padding: 15px; border-radius: 5px; border-left: 4px solid #ffc107;">
                <strong>⚠️ IMPORTANT DISCLAIMER:</strong><br>
                This is AI-generated information for educational purposes only. 
                It does not constitute legal advice. Laws are complex and fact-specific.
                Please consult a qualified lawyer licensed in your jurisdiction before taking any legal action.
            </div>
        </div>
        """
        
        return html

# Initialize the advisor
advisor = IndianLegalAdvisor()
print("\n" + "="*60)
print("Indian Legal Advisor ready to answer your queries!")
print("="*60)


### 4.4 Test the System

Let's test with a sample query to verify everything works.


In [ ]:
# Test with a sample query
test_query = "My employer fired me without notice after 3 years of service. What are my rights?"

print(f"Test Query: {test_query}\n")
print("="*70)

result = advisor.process_query(test_query)

# Display search statistics
print("\n📊 SEARCH RESULTS:\n")

if result['matching_qa']:
    print(f"✓ Found {len(result['matching_qa'])} matching Q&A pairs")
    print(f"  Best match similarity: {result['matching_qa'][0]['similarity']:.3f}")

if result['relevant_statutes']:
    print(f"✓ Found {len(result['relevant_statutes'])} relevant statutes")
    for stat in result['relevant_statutes'][:2]:
        print(f"    - {stat['act']}, Section {stat['section']}")

if result['relevant_cases']:
    print(f"✓ Found {len(result['relevant_cases'])} relevant cases")
    print(f"  Best case relevance: {result['relevant_cases'][0]['relevance_score']:.3f}")
    for case in result['relevant_cases'][:2]:
        print(f"    - {case['case_name']} {case['citation']}")

print("\n" + "="*70)
print("\n📋 LEGAL ADVICE:\n")
print(result['advice'])


---
<a id="section5"></a>
## 5.0 💬 Interactive Query Interface

Now let's create an interactive interface using ipywidgets for easy querying.


In [ ]:
class InteractiveLegalAssistant:
    """
    Interactive UI for the Legal Advisor using ipywidgets
    """
    
    def __init__(self, advisor):
        self.advisor = advisor
        self.conversation_history = []
        self.create_ui()
    
    def create_ui(self):
        """Create the interactive user interface"""
        # Title header
        self.title = widgets.HTML(
            value="""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        padding: 25px; border-radius: 10px; margin-bottom: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                <h2 style="color: white; margin: 0; text-align: center; font-size: 28px;">
                    🏛️ Indian Legal AI Advisor
                </h2>
                <p style="color: white; margin: 8px 0 0 0; text-align: center; font-size: 16px;">
                    Ask your legal questions in natural language • Get instant information with citations
                </p>
            </div>
            """
        )
        
        # Query input textarea
        self.query_box = widgets.Textarea(
            placeholder='Example: "Can I file a complaint if a product I bought online is defective?"',
            description='Your Question:',
            layout=widgets.Layout(width='95%', height='120px'),
            style={'description_width': '120px'}
        )
        
        # Domain filter dropdown
        self.domain_filter = widgets.Dropdown(
            options=['All Domains', 'Consumer Law', 'Property Law', 'Labor Law', 'Criminal Law', 'Family Law', 'Constitutional Law'],
            value='All Domains',
            description='Focus Area:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        # Action buttons
        self.submit_btn = widgets.Button(
            description='🔍 Get Legal Information',
            button_style='primary',
            layout=widgets.Layout(width='220px', height='45px'),
            style={'button_color': '#2196F3', 'font_weight': 'bold'}
        )
        self.submit_btn.on_click(self.on_submit)
        
        self.clear_btn = widgets.Button(
            description='🗑️ Clear',
            button_style='warning',
            layout=widgets.Layout(width='120px', height='45px')
        )
        self.clear_btn.on_click(self.on_clear)
        
        # Output area
        self.output = widgets.Output(
            layout=widgets.Layout(width='95%', min_height='200px', border='1px solid #ddd', 
                                 padding='10px', border_radius='5px')
        )
        
        # Sample queries
        self.samples = widgets.HTML(
            value="""
            <div style="background: #e3f2fd; padding: 20px; border-radius: 8px; margin-top: 20px;">
                <h4 style="margin-top: 0; color: #1976d2;">📝 Sample Questions You Can Ask:</h4>
                <ul style="margin: 10px 0; line-height: 1.8;">
                    <li><strong>Consumer:</strong> "Can I file a complaint if a product I bought online is defective?"</li>
                    <li><strong>Property:</strong> "My neighbor built a wall blocking sunlight. Is this legal?"</li>
                    <li><strong>Employment:</strong> "Can my employer terminate me without notice?"</li>
                    <li><strong>Property:</strong> "Seller didn't tell me about property defects. Can I cancel the sale?"</li>
                    <li><strong>Criminal:</strong> "What is IPC Section 420 and when does it apply?"</li>
                    <li><strong>Civil:</strong> "My landlord is not returning my security deposit. What can I do?"</li>
                    <li><strong>Property:</strong> "Builder delayed giving me possession. Can I claim compensation?"</li>
                    <li><strong>Family:</strong> "What should I do if facing domestic violence?"</li>
                </ul>
            </div>
            """
        )
        
        # Assemble UI
        self.ui = widgets.VBox([
            self.title,
            self.query_box,
            self.domain_filter,
            widgets.HBox([self.submit_btn, self.clear_btn], layout=widgets.Layout(margin='10px 0')),
            self.output,
            self.samples
        ])
        
        display(self.ui)
    
    def on_submit(self, button):
        """Handle submit button click"""
        query = self.query_box.value.strip()
        
        if not query:
            with self.output:
                clear_output()
                display(HTML("""
                <div style="padding: 20px; background: #fff3cd; border-radius: 5px; border-left: 4px solid #ffc107;">
                    ⚠️ Please enter a legal question.
                </div>
                """))
            return
        
        with self.output:
            clear_output()
            display(HTML("""
            <div style="padding: 20px; background: #e3f2fd; border-radius: 5px; text-align: center;">
                <div style="font-size: 18px; color: #1976d2;">⚖️ Analyzing your query and searching legal database...</div>
                <div style="margin-top: 10px; color: #666;">This may take a few seconds...</div>
            </div>
            """))
        
        # Process query
        try:
            result = self.advisor.process_query(query)
            
            # Store in history
            self.conversation_history.append({
                'query': query,
                'result': result,
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            
            # Display result
            with self.output:
                clear_output()
                html_response = self.advisor.format_response_html(result)
                display(HTML(html_response))
                
        except Exception as e:
            with self.output:
                clear_output()
                display(HTML(f"""
                <div style="padding: 20px; background: #ffebee; border-radius: 5px; border-left: 4px solid #f44336;">
                    ❌ <strong>Error:</strong> {str(e)}<br>
                    Please try again or rephrase your question.
                </div>
                """))
    
    def on_clear(self, button):
        """Clear the output and input"""
        self.query_box.value = ''
        with self.output:
            clear_output()

# Create and display interactive assistant
print("\n" + "="*70)
print("Starting Interactive Legal Assistant...")
print("="*70 + "\n")

interactive_assistant = InteractiveLegalAssistant(advisor)


---
<a id="section6"></a>
## 6.0 📖 Sample Use Cases

Let's demonstrate the system with real-world legal scenarios that Indians commonly face.

### Use Case 1: Consumer Dispute


In [ ]:
# Consumer dispute example
consumer_query = """I bought a smartphone online that stopped working after 2 months. 
The company is not responding to my emails and refuses to replace it. What can I do?"""

print("="*70)
print("USE CASE 1: CONSUMER DISPUTE - Defective Product")
print("="*70)
print(f"\nQuery: {consumer_query}\n")

result = advisor.process_query(consumer_query)
display(HTML(advisor.format_response_html(result)))


### Use Case 2: Property Dispute


In [ ]:
# Property dispute example
property_query = """My neighbor built a wall that blocks sunlight to my house. 
I have been getting sunlight for 25 years. Is this legal? What can I do?"""

print("="*70)
print("USE CASE 2: PROPERTY DISPUTE - Easement of Light")
print("="*70)
print(f"\nQuery: {property_query}\n")

result = advisor.process_query(property_query)
display(HTML(advisor.format_response_html(result)))


### Use Case 3: Employment Issue


In [ ]:
# Employment issue example
employment_query = """My company terminated me without notice yesterday. 
I had been working there for 3 years and 4 months. Is this allowed? What are my rights?"""

print("="*70)
print("USE CASE 3: EMPLOYMENT ISSUE - Illegal Termination")
print("="*70)
print(f"\nQuery: {employment_query}\n")

result = advisor.process_query(employment_query)
display(HTML(advisor.format_response_html(result)))


### Use Case 4: Criminal Law Query


In [ ]:
# Criminal law example
criminal_query = """Someone cheated me in a fake investment scheme and took ₹5 lakhs. 
They promised 30% monthly returns but now disappeared. What legal action can I take?"""

print("="*70)
print("USE CASE 4: CRIMINAL LAW - Investment Fraud")
print("="*70)
print(f"\nQuery: {criminal_query}\n")

result = advisor.process_query(criminal_query)
display(HTML(advisor.format_response_html(result)))


---
<a id="section7"></a>
## 7.0 ⚠️ Ethical Safeguards & Disclaimers

### Important Legal Disclaimers

**⚖️ THIS SYSTEM PROVIDES INFORMATION, NOT LEGAL ADVICE**

#### What This System IS:
✅ An educational tool for learning about Indian laws  
✅ A search engine for legal information  
✅ A starting point for understanding your rights  
✅ A guide to help you ask better questions to lawyers  

#### What This System IS NOT:
❌ A substitute for a qualified lawyer  
❌ Legal advice for your specific situation  
❌ A tool for making legal decisions  
❌ A replacement for professional legal consultation  

### Critical Understanding

1. **No Attorney-Client Relationship**: Using this system does not create any attorney-client relationship or privilege.

2. **Educational Purpose Only**: All information is for educational purposes and general awareness about Indian laws.

3. **Fact-Specific Nature**: Legal outcomes depend heavily on specific facts, evidence, jurisdiction, and current law. This system provides general information only.

4. **Laws Change**: Indian laws are amended regularly. Information may become outdated.

5. **Jurisdictional Variations**: Laws and procedures vary by state in India (e.g., property laws, local regulations).

6. **No Liability**: Creators assume NO liability for actions taken based on AI-generated information.

### When You MUST Consult a Lawyer

🚨 **Immediately consult a qualified lawyer if:**

- **Criminal Matters**: Any criminal charge, FIR, arrest, investigation
- **Court Proceedings**: You need to file a case or defend yourself
- **Large Financial Stakes**: Property disputes, business matters, inheritance
- **Time-Sensitive**: Limitation periods running out, urgent legal action needed
- **Rights at Risk**: Personal liberty, custody, deportation, property loss
- **Complex Situations**: Multiple laws, unclear facts, precedent interpretation
- **Serious Consequences**: Imprisonment, heavy fines, loss of rights

### Ethical AI Principles

This system follows these principles:

1. **Transparency**: Clear that information is AI-generated
2. **Accuracy**: Based on real legal principles (with synthetic demo data)
3. **Accessibility**: Making legal information available to all
4. **Safety**: Always recommending professional consultation
5. **Privacy**: Not storing personal information
6. **Non-Discrimination**: Equal access regardless of background
7. **Harm Prevention**: Warnings for sensitive matters

### Limitations of This System

**Technical Limitations:**
- Small synthetic dataset (demo version)
- No real-time legal database integration
- Basic search algorithms
- English language only
- No document analysis capability

**Legal Limitations:**
- Cannot analyze your specific documents
- Cannot predict case outcomes
- Cannot represent you in court
- Cannot provide strategic legal advice
- Cannot file cases or legal documents for you

**Practical Limitations:**
- No verification of facts you provide
- Cannot assess credibility of evidence
- Cannot negotiate on your behalf
- Cannot attend court hearings
- Cannot cross-examine witnesses

### How to Use This System Responsibly

✅ **DO:**
- Use for general legal awareness
- Learn about applicable laws
- Understand basic legal concepts
- Prepare better questions for your lawyer
- Know when to seek legal help
- Gather relevant information

❌ **DON'T:**
- Make legal decisions based solely on AI output
- File legal cases without lawyer consultation
- Miss limitation periods trusting AI advice
- Ignore specific legal advice from qualified lawyers
- Share confidential/sensitive personal information
- Use for actual legal matters without verification

### Resources for Free/Low-Cost Legal Help

If you cannot afford a lawyer:

1. **District Legal Services Authority (DLSA)**: Free legal aid in every district
2. **State Legal Services Authority**: Free legal consultation
3. **National Legal Services Authority (NALSA)**: Coordinate free legal aid
4. **Lok Adalat**: Alternative dispute resolution - faster, cheaper
5. **Law Universities**: Free legal aid clinics
6. **Pro Bono Lawyers**: Many lawyers offer free initial consultation
7. **Legal Aid Helplines**: State-specific toll-free numbers

**Contact:**
- NALSA Website: https://nalsa.gov.in/
- DLSA: Visit your district headquarters
- Women Helpline: 1091
- National Commission for Women: 011-26942369

### Your Responsibility

By using this system, you acknowledge that:
- You understand this is not legal advice
- You will consult a qualified lawyer for actual legal matters
- You will verify all information independently
- You will not rely solely on AI-generated information for legal decisions
- You understand the creators bear no liability for your use of this system


In [ ]:
# Ethical Safeguards Implementation
class EthicalSafeguards:
    """
    Class to implement ethical safeguards and warnings
    """
    
    SENSITIVE_TOPICS = [
        'death penalty', 'capital punishment', 'terrorism', 'sedition',
        'national security', 'rape', 'murder', 'child abuse', 
        'human trafficking', 'assassination', 'bomb'
    ]
    
    HIGH_STAKES_KEYWORDS = [
        'court', 'trial', 'arrest', 'warrant', 'bail', 'custody',
        'divorce', 'will', 'property sale', 'conviction', 'sentence',
        'deportation', 'extradition', 'confession'
    ]
    
    @staticmethod
    def check_query_sensitivity(query: str) -> Dict[str, any]:
        """
        Analyze query for sensitivity and provide appropriate warnings
        """
        query_lower = query.lower()
        
        # Check for sensitive topics
        is_sensitive = any(topic in query_lower for topic in EthicalSafeguards.SENSITIVE_TOPICS)
        
        # Check for high-stakes matters
        is_high_stakes = any(keyword in query_lower for keyword in EthicalSafeguards.HIGH_STAKES_KEYWORDS)
        
        return {
            'is_sensitive': is_sensitive,
            'is_high_stakes': is_high_stakes,
            'warning_level': 'CRITICAL' if is_sensitive else ('HIGH' if is_high_stakes else 'STANDARD'),
            'recommendation': EthicalSafeguards._get_recommendation(is_sensitive, is_high_stakes)
        }
    
    @staticmethod
    def _get_recommendation(is_sensitive: bool, is_high_stakes: bool) -> str:
        """
        Get appropriate recommendation based on query analysis
        """
        if is_sensitive:
            return ("🚨 CRITICAL: This matter involves very serious legal issues. "
                   "You MUST consult a qualified criminal lawyer IMMEDIATELY. "
                   "Do not rely on AI for such sensitive and serious matters. "
                   "Contact your District Legal Services Authority for free legal aid if needed.")
        elif is_high_stakes:
            return ("⚠️ HIGH PRIORITY: This appears to be a high-stakes legal matter "
                   "with significant consequences. While we provide general information, "
                   "you MUST consult a qualified lawyer before taking any action. "
                   "Do not make decisions based solely on AI information.")
        else:
            return ("ℹ️ STANDARD: For specific legal advice tailored to your situation, "
                   "please consult a qualified lawyer. This information is for general awareness only.")
    
    @staticmethod
    def add_disclaimer_to_response(response: str, query: str) -> str:
        """
        Add appropriate disclaimer based on query sensitivity
        """
        check = EthicalSafeguards.check_query_sensitivity(query)
        
        disclaimer = f"""

---

### ⚖️ Legal Disclaimer

**Warning Level: {check['warning_level']}**

{check['recommendation']}

**Remember:** This AI-generated information is for educational purposes only and does not 
constitute legal advice. Laws are complex and fact-specific. Always consult a qualified 
lawyer licensed in your jurisdiction before taking any legal action.
        """
        
        return response + disclaimer

# Test ethical safeguards
print("\n" + "="*70)
print("Testing Ethical Safeguards")
print("="*70 + "\n")

test_queries = [
    "What is IPC Section 420?",  # Normal query
    "I need to file for bail in a court case",   # High stakes
    "How to defend against murder charges in court"  # Sensitive
]

for query in test_queries:
    check = EthicalSafeguards.check_query_sensitivity(query)
    print(f"Query: {query}")
    print(f"  Warning Level: {check['warning_level']}")
    print(f"  Sensitive: {check['is_sensitive']}, High Stakes: {check['is_high_stakes']}")
    print(f"  Recommendation: {check['recommendation'][:100]}...")
    print()


---

## 🎯 Summary & Conclusion

### What We've Built

This notebook demonstrates a **comprehensive Indian Legal AI Advisor** with:

✅ **Legal Database**: Synthetic dataset of Indian cases (10 cases), statutes (16 statutes), and Q&A pairs (13 pairs)  
✅ **Semantic Search**: Finding relevant cases using sentence transformers and FAISS  
✅ **Statute Lookup**: Keyword-based search for applicable laws  
✅ **Q&A Matching**: Direct answers to common legal questions  
✅ **Interactive Interface**: User-friendly query system with ipywidgets  
✅ **Ethical Safeguards**: Comprehensive disclaimers and warnings  

### System Capabilities (Phase 1)

**Current Features:**
- Answer common legal queries about Indian law in English
- Search case law database for relevant precedents with semantic matching
- Identify applicable statutes and sections automatically
- Provide general legal information with proper citations
- Interactive query interface with HTML formatted responses
- Ethical warnings for sensitive/high-stakes matters
- Sample use cases demonstrating real-world scenarios

### Limitations (This Version)

**Not Included:**
- Document analysis (contracts, notices, petitions)
- Document generation (legal notices, drafts)
- Multi-lingual support (Hindi, regional languages)
- Integration with live legal databases (Indian Kanoon, IndiaCode)
- Advanced RAG with large language models (GPT-4, Claude)
- Voice interface
- Real-time case law updates
- Predictive analytics for case outcomes

### Technology Stack

- **NLP**: sentence-transformers (all-MiniLM-L6-v2)
- **Vector Search**: FAISS (Facebook AI Similarity Search)
- **UI**: ipywidgets for Jupyter notebooks
- **Data**: NumPy, Pandas for data processing
- **Python**: 3.9+ with type hints

### Future Enhancements (Roadmap)

**Phase 2** - Enhanced Features:
- Hindi language support using translation models
- Larger case law database (1000+ cases)
- Integration with IndiaCode.nic.in for live statute updates
- PDF contract analysis
- Legal document risk assessment

**Phase 3** - Advanced AI:
- GPT-4/Claude integration for better advice generation
- Multi-hop reasoning across multiple cases
- Citation network visualization
- Precedent analysis and prediction
- Automated legal research summaries

**Phase 4** - Production Deployment:
- REST API with FastAPI
- User authentication and query history
- Lawyer consultation marketplace integration
- Mobile app (Android/iOS)
- Regional language support (10+ languages)
- Voice interface
- Integration with e-courts platform

### Impact Potential

🎯 **Accessibility**: Makes legal information available to 1.3 billion Indians  
⚖️ **Legal Awareness**: Educates citizens about their rights and laws  
💰 **Cost Savings**: Saves ₹2,000-10,000 per preliminary consultation  
⏱️ **Time Savings**: Instant access vs weeks for lawyer appointment  
🌍 **Social Good**: Improves access to justice, especially for underserved populations  

### How to Use This Notebook

1. **Run all cells** in sequence from top to bottom
2. **Wait** for models to load (sentence transformer download may take 1-2 minutes first time)
3. **Use the interactive interface** in Section 5.0 to ask legal questions
4. **Try sample queries** provided or ask your own questions
5. **Review responses** with citations and applicable laws
6. **Always consult a lawyer** for actual legal matters

### Educational Value

This system demonstrates:
- NLP and semantic search in legal domain
- Building domain-specific AI assistants
- Retrieval-Augmented Generation (RAG) architecture
- Ethical AI development principles
- Human-AI collaboration in sensitive domains
- Practical application of ML for social good

### Acknowledgments

- **Indian Legal System**: Based on Indian laws and legal principles
- **Transformers**: HuggingFace for sentence-transformers
- **FAISS**: Facebook AI for efficient similarity search
- **Open Source Community**: For all the amazing libraries

---

## 🙏 Thank You!

This Indian Legal AI Advisor is a step toward **democratizing legal knowledge** and **improving access to justice** in India.

**Remember**: This is an educational tool. For actual legal matters, always consult qualified legal professionals.

---

**Version**: 1.0 - Basic Query System  
**Date**: February 2026  
**License**: MIT (Educational purposes)  
**Disclaimer**: Educational purposes only. Not legal advice.

**Contact for Issues/Improvements**: [Add your contact if deploying publicly]

---

### 📚 Additional Resources

**Indian Legal Resources:**
- Indian Kanoon: https://indiankanoon.org/
- IndiaCode: https://www.indiacode.nic.in/
- Supreme Court of India: https://main.sci.gov.in/
- National Legal Services Authority: https://nalsa.gov.in/

**Free Legal Aid:**
- District Legal Services Authority in your district
- State Legal Services Authority
- Women Helpline: 1091
- National Commission for Women: 011-26942369

**Learning Resources:**
- NCERT Law Books
- Indian Law textbooks
- Online legal courses (Coursera, edX)

---

**End of Notebook**
